<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/RLM_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://arxiv.org/html/2512.24601v1

## RLM

In [1]:
import openai
import os
import re
import ast
from concurrent.futures import ThreadPoolExecutor
from google.colab import userdata

# --- 1. SETUP ---
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
client = openai.OpenAI()

stats = {"root_tokens": 0, "worker_tokens": 0}

# --- 2. THE RECURSIVE ENGINE ---
def worker_agent(text_slice, task_instruction):
    """The Sub-Agent extracting the actual data."""
    print(f"  [Worker] Reading slice ({len(text_slice)} chars)...")
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a precise data extractor. Return ONLY the answer."},
            {"role": "user", "content": f"TEXT: {text_slice}\nTASK: {task_instruction}"}
        ],
        temperature=0
    )
    stats["worker_tokens"] += resp.usage.total_tokens
    return resp.choices[0].message.content.strip()

def run_parallel_worker(tasks_list):
    """Executes worker calls in parallel threads."""
    print(f"--- [System] Launching {len(tasks_list)} Parallel Workers ---")
    with ThreadPoolExecutor() as executor:
        return list(executor.map(lambda p: worker_agent(*p), tasks_list))

def run_rlm_production(query, context):
    # PHASE 1: INDEXING
    print("--- [Root] Phase 1: Indexing ---")
    index_prompt = "Identify section indices. Output ONLY a Python dict. Format: {'SEC_A': [0, 50], ...}"
    idx_resp = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": f"CONTEXT: {context}\n\n{index_prompt}"}],
        temperature=0
    )
    stats["root_tokens"] += idx_resp.usage.total_tokens

    # Securely extract the dictionary
    idx_text = re.search(r"\{.*\}", idx_resp.choices[0].message.content, re.DOTALL).group(0)
    index_dict = ast.literal_eval(idx_text)
    print(f"--- [System] Index Map Created: {index_dict} ---")

    # PHASE 2: ORCHESTRATION
    env = {
        "CONTEXT": context,
        "batch_worker": run_parallel_worker,
        "INDEX_DICT": index_dict,
        "final_answer": None
    }

    system_instructions = """
    You are a Python Orchestrator.
    Available: 'INDEX_DICT' (dict), 'CONTEXT' (str), and 'batch_worker(tasks)' (func).

    TASK:
    1. Loop through INDEX_DICT. If a key relates to the user query, get its [start, end].
    2. Slice the CONTEXT: text_slice = CONTEXT[start:end]
    3. Create tasks: tasks.append((text_slice, "Extract the secret code"))
    4. Call results = batch_worker(tasks)
    5. final_answer = " ".join(results)

    RULES:
    - NO redefinitions. NO chat.
    - Match keys loosely (e.g., 'SEC_A' matches 'A').
    - Output ONLY ```python ... ```
    """

    print("--- [Root] Phase 2: Orchestrating Workers ---")
    plan_resp = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": system_instructions}, {"role": "user", "content": query}],
        temperature=0
    )
    stats["root_tokens"] += plan_resp.usage.total_tokens

    code_match = re.search(r"```python\s*(.*?)\s*```", plan_resp.choices[0].message.content, re.DOTALL)
    clean_code = code_match.group(1).strip()

    print(f"--- [Root] Executing Plan ---\n{clean_code}\n")
    exec(clean_code, env, env)

    return env.get("final_answer")

# --- 3. RUN ---
huge_context = """SECTION A: The secret code is ALPHA-99.
SECTION B: This is just empty filler.
SECTION C: The secret code is OMEGA-00."""

result = run_rlm_production("Extract codes from A and C.", huge_context)

print(f"\n--- [Final Result] ---\n{result}")
print(f"\n--- [Token Usage] ---\nRoot: {stats['root_tokens']} | Workers: {stats['worker_tokens']}")

--- [Root] Phase 1: Indexing ---
--- [System] Index Map Created: {'SEC_A': [0, 50], 'SEC_B': [51, 80], 'SEC_C': [81, 120]} ---
--- [Root] Phase 2: Orchestrating Workers ---
--- [Root] Executing Plan ---
tasks = []

for key, (start, end) in INDEX_DICT.items():
    if 'A' in key or 'C' in key:
        text_slice = CONTEXT[start:end]
        tasks.append((text_slice, "Extract the secret code"))

results = batch_worker(tasks)
final_answer = " ".join(results)

--- [System] Launching 3 Parallel Workers ---
  [Worker] Reading slice (50 chars)...
  [Worker] Reading slice (29 chars)...
  [Worker] Reading slice (36 chars)...

--- [Final Result] ---
ALPHA-99 There is no secret code in the provided text. OMEGA-00

--- [Token Usage] ---
Root: 355 | Workers: 148


The final output of the code demonstrates a successful execution of the Recursive Language Model (RLM) framework to extract specific information from a structured text.

Here is a summary of the results:

* **Extraction Results**: The system successfully identified and retrieved the secret codes "ALPHA-99" from SECTION A and "OMEGA-00" from SECTION C.
* **Intelligent Filtering**: The worker agent correctly identified that there was "no secret code in the provided text" for the slice corresponding to SECTION B, demonstrating that the sub-agents can validate content rather than just returning raw strings.
* **Parallel Execution**: The orchestrator launched three parallel workers simultaneously, reducing the total latency required to process multiple document sections.
* **Token Efficiency**:
* **Root Model (Logic)**: Used 355 tokens to index the context and generate the Python execution plan.
* **Worker Models (Data)**: Used 148 tokens to perform the actual reading and extraction.
* **Total Impact**: The high-cost model (Root) focused only on structural logic, while the low-cost models (Workers) handled the data-heavy tasks, optimizing the overall cost of the operation.

## Agentic RLM Demonstration (One Cell Code)

In [4]:
import openai
import os
import re
import json
from concurrent.futures import ThreadPoolExecutor
from google.colab import userdata

# --- 1. SETUP ---
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
client = openai.OpenAI()
stats = {"root_tokens": 0, "worker_tokens": 0}

# --- 2. THE RECURSIVE ENGINE ---

def worker_agent(text_slice, task_instruction):
    """Sub-Agent: Precise data extractor (Cheap Model)."""
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Extract requested facts accurately. If not found, say 'NONE'."},
            {"role": "user", "content": f"TEXT: {text_slice}\nTASK: {task_instruction}"}
        ],
        temperature=0
    )
    stats["worker_tokens"] += resp.usage.total_tokens
    return resp.choices[0].message.content.strip()

def orchestrator(tasks_list):
    """Tool: Parallel worker launcher."""
    if not tasks_list: return []
    print(f"--- [System] Launching {len(tasks_list)} Parallel Workers ---")
    with ThreadPoolExecutor() as executor:
        return list(executor.map(lambda p: worker_agent(*p), tasks_list))

def final_synthesizer(raw_results, original_query):
    """Tool: Final cleanup and synthesis (Smart Model)."""
    print("--- [Agent] Phase 3: Consolidating fragments... ---")
    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": f"Query: {original_query}\nFragments: {raw_results}\n\nCombine these into one clean, professional sentence. Ignore 'NONE'."}],
        temperature=0
    )
    stats["root_tokens"] += resp.usage.total_tokens
    return resp.choices[0].message.content

def run_master_rlm(query, context):
    # PHASE 1: INDEXING
    print("--- [Agent] Phase 1: Indexing context... ---")
    index_prompt = "Return a JSON object of section boundaries. Format: {'SEC_A': [0, 50], ...}. Ensure values are lists of TWO integers."
    idx_resp = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": f"CONTEXT: {context}\n\n{index_prompt}"}]
    )
    stats["root_tokens"] += idx_resp.usage.total_tokens

    idx_match = re.search(r"\{.*\}", idx_resp.choices[0].message.content, re.DOTALL)
    index_dict = json.loads(idx_match.group(0))

    # PHASE 2: EXECUTION (Slicing logic fix)
    env = {
        "CONTEXT": context,
        "batch_worker": orchestrator,
        "INDEX_DICT": index_dict,
        "consolidate": final_synthesizer,
        "QUERY": query,
        "final_answer": "No answer generated."
    }

    logic_prompt = """
    You are an Agentic RLM Orchestrator.
    Variable 'INDEX_DICT' contains lists of [start, end].

    INSTRUCTIONS:
    1. Loop through INDEX_DICT.values().
    2. Slice correctly: CONTEXT[val[0]:val[1]].
    3. Call results = batch_worker([(slice, "Extract info") for slice in slices]).
    4. final_answer = consolidate(results, QUERY)

    OUTPUT ONLY VALID PYTHON.
    """

    print("--- [Agent] Phase 2: Orchestrating Workers ---")
    plan_resp = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": logic_prompt}, {"role": "user", "content": query}]
    )
    stats["root_tokens"] += plan_resp.usage.total_tokens

    code_match = re.search(r"```python\s*(.*?)\s*```", plan_resp.choices[0].message.content, re.DOTALL)
    code_block = code_match.group(1).strip() if code_match else plan_resp.choices[0].message.content

    print(f"--- [Agent] Executing Script ---\n{code_block}\n")
    try:
        exec(code_block, env, env)
    except Exception as e:
        return f"Runtime Error: {e}"

    return env.get("final_answer")

# --- 3. RUN ---
massive_document = """
SECTION_ALPHA: The decryption key is 'X-ray-99'. Security level: High.
SECTION_BETA: Redundant logs...
SECTION_GAMMA: The primary administrator is 'Alice Smith'. Access: Full.
"""

final_ans = run_master_rlm("Find the decryption key and the administrator's name.", massive_document)
print(f"--- [FINAL RESULT] ---\n{final_ans}")

--- [Agent] Phase 1: Indexing context... ---
--- [Agent] Phase 2: Orchestrating Workers ---
--- [Agent] Executing Script ---
# INDEX_DICT is assumed to be provided, as CONTEXT and QUERY
INDEX_DICT = {
    'decryption_key': [0, 50],
    'admin_name': [51, 100]
}

# Slices the context based on the INDEX_DICT
slices = [CONTEXT[val[0]:val[1]] for val in INDEX_DICT.values()]

# Calls the batch_worker function with each slice and "Extract info" instruction
results = batch_worker([(slice, "Extract info") for slice in slices])

# Consolidates the results and query to produce the final answer
final_answer = consolidate(results, QUERY)

--- [System] Launching 2 Parallel Workers ---
--- [Agent] Phase 3: Consolidating fragments... ---
--- [FINAL RESULT] ---
The decryption key is X-ray-99.
